# Backtracking

Backtracking is a way of incrementally building a solution to a problem with recursion. The algorithms repeatedly cycles through possible extensions to the partial solution, rejecting and trying the next solution if it finds the partial solution cannot be part of a solution, until it finds that the partial solution solves the original problem and outputs this solution. Coding backtracking can be done using recursion. 

The backtrack function has 5 functional inputs: reject_func, accept_func, first_func, next_func, and output_func which are determined by the nature of the problem. The input n is the data for the particular problem, for example it may be the dimension of the problem, and the input c is the current partial solution. To solve a problem, one should call backtrack with c as the "root" solution, i.e the root from which any solution must be an extension. In both examples below, the root solution can be NULL.

In [1]:
backtrack <- function(reject_func, accept_func, first_func, 
                      next_func, output_func, c = NULL, n=NULL){
  if (reject_func(n, c)) {
    return()
  }
  if (accept_func(n, c)) {
    output_func(n, c)
  }
  s <- first_func(n, c)
  while(!is.null(s)) {
    backtrack(reject_func, accept_func, first_func, next_func, output_func,
              s, n=n)
    s <- next_func(n, s) 
  }
  return(backtrack_solution)
}

# Partitions

In order to find all integer partitions of n, one can use backtracking. This can be done by considering partial partitions of n. A partition can be created by choosing the largest part to be some i between 1 and n, and then recursively choosing the next largest part to be between 1 and i etc. In order to find all partitions, the code simply cycles through all the options. Backtracking is used to reject potential partitions if their sum is larger than n. Similarly, potential partitions are accepted and added to the list of solutions (by the output function) if their sum to n. It makes sense to output a list, since the partitions are vectors of different lengths.

In [2]:
#Function to reject the partition and stop adding to it, if it sums to more than n.
part_reject <- function(n, partition){
  if (sum(partition) > n){
    return(TRUE)
  }
  return(FALSE)
}

#Function to accept the partition if it sums to n
part_accept <- function(n, partition){
  if (sum(partition) == n){
    return(TRUE)
  }
  return(FALSE)
}

#Function to add accepted partitions to a backtrack_solution global variable so it can be returned
part_output <- function(n, partition){
  if (length(partition) == n){ 
    #The first partition found will be length n (and no other partitions will be), so create the backtrack_solution global variable now
    backtrack_solution <<- list(partition)
  }
  else{
    backtrack_solution <<- append(backtrack_solution, list(partition))
  }
}

#Function for the first extension to a partial partition - appending 1 to the partial partition.
part_first <- function(n, partition){
  i <<- 1
  return(c(partition, 1))
}

#Function for subsequent extensions to a partial partition - appending i to the partial partition
part_next <- function(n, partition){
  i <<- i + 1
  if (i > n){
    return(NULL)
  }
  new_part <- partition[ partition >= i ] #Remove the previous extensions of the partial partition, or start a new partition
  return(c(new_part, i))
}

### Examples of Using Code to Find Partitions

In [3]:
#A simple but informative example
backtrack(part_reject, part_accept, part_first, part_next, part_output,
                 n = 6)

[[1]]
[1] 1 1 1 1 1 1

[[2]]
[1] 2 1 1 1 1

[[3]]
[1] 2 2 1 1

[[4]]
[1] 2 2 2

[[5]]
[1] 3 1 1 1

[[6]]
[1] 3 2 1

[[7]]
[1] 3 3

[[8]]
[1] 4 1 1

[[9]]
[1] 4 2

[[10]]
[1] 5 1

[[11]]
[1] 6

There are eleven partitions of 6, as expected.

The code works quickly up to n = 20, generally taking no more than 0.1 seconds on my device. This suggests the code is robust for small n. My code is able to run quickly as none of the backtracking functions take a long time to compute; the main operations used are summing, appending to and checking values of a list for a condition. This is efficient because due to the reject function, the lists in question will never be longer than n+1 items. As values are added in decreasing order, it is not neccessary to use costly functions like "sort"

However, on my device, I encounter a "node stack overflow" error for n > 20. This is because there is a high degree of recursion; the backtracking function must be called for every number added to every partition.

In [4]:
#The code can work quickly and efficiently for up to n=20 on this computer.
tik <- Sys.time()
part20 <- backtrack(part_reject, part_accept, part_first, part_next, part_output,
                 n = 20)
tok <- Sys.time()
print(tok - tik)

Time difference of 0.0672071 secs


# Gray Codes

In order to find Gray Codes using a backtracking approach, the code builds up a Gray code by adding new codewords to the code until it has an appropriate Gray Code. 

It does this using the "first" and "next" functions to extend the code by adding codewords to the end. These extension are found by switching a single bit of the final codeword to generate the next codeword. This guarantees that the new codeword has a Hamming distance of one from the codeword before it, however it may already be in the code. The reject function checks whether the new codeword is a duplicate of any previous codewords; if so, the code is rejected. If the code is not rejected, and therefore contains no duplicates, the accept function is called, which checks whether the code is of length (2^n) - if so, all possible codewords have been found and the code is a Gray Code. This is an efficient flow, ensuring that conditions are not checked unnecessarily, and that the same condition isn't checked twice on the same code. Finally, the output function assigns the full Gray Code to the "backtrack_solution" global variable, so that the backtracking function can return it.

In my code, I chose for the first extension to switch the rightmost bit, and for the each following extension, the next bit to the left is switched.
This ensures that starting from the all-zero codeword, my code generated the standard Gray Code; also the deterministic nature of this means my code
is repeatable. However it might reduce recursion if the order that the bits were switched was randomised. For example, using my method, the
first extension to the second codeword will always be the first codeword again, so it will be rejected, however this would be less likely with a
randomised choice of which bit to flip, leading to fewer rejections and thus less recursion would be needed. Reducing recursion would mean it was possible to use backtracking to find longer Gray Codes, since in order to prevent infinite recursion from stalling a computer, there is a limit of the number of nested expressions a computer will evaluate. 

The Gray Code is returned as a matrix, with each row corresponding to a codeword. The j-th row corresponds to the codeword for (j - 1); e.g. the first row is the codeword for 0. This is a more appropriate data structure than returning a list as each codeword should be the same length. However, another improvement could be to use bit operators instead of a matrix.

If just n is specified, then the code will return the Standard Gray Code of length n. Otherwise, by specifying c, as a matrix of codewords, you can determine the first codeword in the code, and the code will return a Gray Code starting with the specified codeword.


In [5]:
# Function which rejects if the code is too long, or the new codeword is a duplicate of a previous codeword
gray_reject <- function(n = NULL, partial_code){
  if(is.null(partial_code)){
    return(FALSE)
  }
  n <- dim(partial_code)[2]
  length <- dim(partial_code)[1]
  if(length > 2^n) {  
    return (TRUE)
  }
  if (length > 1) {
    for (i in 1:(length-1)) {
      if (sum(partial_code[i,] != partial_code[length,]) == 0){
        return(TRUE)
      }
    }
  }
  return(FALSE)
}

# Function which accepts if the code is the right length (since gray_reject ensures there are no duplicates)
gray_accept <- function(n = NULL, code){
  n <- dim(code)[2]
  if (sum(dim(code) == c(2^n,n)) != 2) {
    return(FALSE)
  }
  return(TRUE)
}

# Function which assigns the global variable "solution" so the function can output it
gray_output <- function(n = NULL, code){
  backtrack_solution <<- code
}

# Function to flip bits from 0 to 1, or vice-versa
flip <- function(num){abs(num-1)}

# Function to create the first extension of the partial code, by flipping the rightmost bit of the last codeword to create the next codeword
gray_first <- function(n = NULL, partial_code){
  if (is.null(n)){
    n <- dim(partial_code)[2]
  }
  if (is.null(partial_code)){
    partial_code <- matrix(rep(0,n), nrow =1)
  }
  i <<- 0
  vec <- tail(partial_code, 1)
  vec[n] <- flip(vec[n])
  partial_code <- rbind(partial_code , vec)
  return(partial_code)
}

# Function to create the next extension of the partial code, by flipping the next bit to the left
gray_next <- function(n= NULL, partial_code){
  if (is.null(n)){
    n <- dim(partial_code)[2]
  }
  i <<- i + 1
  if (i >= n){ #If all possible extensions have been tried, return NULL
    return(NULL)
  }
  length <- dim(partial_code)[1]
  partial_code[length,(n-i+1)] <- flip(partial_code[length,(n-i+1)]) #This reverses the flip from the previous extension
  partial_code[length,n-i] <- flip(partial_code[length,n-i])
  return(partial_code)
}


### Examples of Using the Code to Find Gray Codes

#### Example - Finding the Standard Gray Code of Length 4

In [6]:
backtrack(gray_reject, gray_accept, gray_first, gray_next, gray_output, n = 4)

,0,0,0,0
"[1,]",0,0,0,1
"[1,]",0,0,1,1
"[1,]",0,0,1,0
"[1,]",0,1,1,0
"[1,]",0,1,1,1
"[1,]",0,1,0,1
"[1,]",0,1,0,0
"[1,]",1,1,0,0
"[1,]",1,1,0,1
"[1,]",1,1,1,1


#### Example - Finding a Gray Code of Length 3 where 0 is represented by (1,1,1).

In [7]:
backtrack(gray_reject, gray_accept, gray_first, gray_next, gray_output, c = matrix(rep(1, 3), nrow =1))

,1,1,1
"[1,]",1,1,0
"[1,]",1,0,0
"[1,]",1,0,1
"[1,]",0,0,1
"[1,]",0,0,0
"[1,]",0,1,0
"[1,]",0,1,1


#### Using Embedding of Gray Codes to Find the Standard Gray Codes of Length 12

On my device, the code is able to compute Gray Codes up to length 11, however for longer lengths, I get a "node stack overflow" error due to the amount of recursion required. It is possible to find Gray Codes of length longer than 11 using a non-backtracking approach, relying on the fact that shorter Gray Codes are embedded in longer Gray Codes. For example, to find a Gray Code of Length 12, I used the following code.

In [8]:
gray_11 <- backtrack(gray_reject, gray_accept, gray_first, gray_next, gray_output, n = 11)
gray_12_half1 <- cbind(rep(0, times = 2^11), gray_11)
gray_12_half2 <- cbind(rep(1, times = 2^11), gray_11[2^11:1,])
gray_12 <- rbind(gray_12_half1, gray_12_half2)
gray_12

,0,0,0,0,0,0,0,0,0,0,0,0
"[1,]",0,0,0,0,0,0,0,0,0,0,0,1
"[1,]",0,0,0,0,0,0,0,0,0,0,1,1
"[1,]",0,0,0,0,0,0,0,0,0,0,1,0
"[1,]",0,0,0,0,0,0,0,0,0,1,1,0
"[1,]",0,0,0,0,0,0,0,0,0,1,1,1
"[1,]",0,0,0,0,0,0,0,0,0,1,0,1
"[1,]",0,0,0,0,0,0,0,0,0,1,0,0
"[1,]",0,0,0,0,0,0,0,0,1,1,0,0
"[1,]",0,0,0,0,0,0,0,0,1,1,0,1
"[1,]",0,0,0,0,0,0,0,0,1,1,1,1


# Conclusion

Using a generic backtrack function where different functions are defined for different problems makes this solution very reusable for any backtracking problem. Although in both the Gray code and partitioning examples, the input n had to be an integer, for more complicated problems, n could be any data structure and could encode more information about the specific problem. This flexibility means my code is reusable for more complex backtracking problems.

On my computer, the code was not able to find Gray Codes longer than 11 or partitions of n > 20. This is because to do so requires more recursion than my computer was set up to do. In RStudio, I was able to manually increase the recursion depth with the command "options(expressions = 500000)". This command sets the maximum number of nested functions which will be evaluated; 500000 is the maximum allowed by Jupyter and RStudio. However, even by setting the recursion depth to 500000, it is not possible to go past n=12 with Gray Codes or n=20 with partitions, using my code.

In order for backtrack to return a solution, I chose to have the output_func assign a global variable "backtrack_solution" to the solution found. I called this variable "backtrack_solution" rather than "solution" because the function will overwrite the value of this variable if it is already assigned, so it is best to use a name which likely won't be used outside this function. Nevertheless, it would be better if outputting could be done without assigning global variables, to avoid overwriting issues and make code using the backtrack function more replicatable and reusable.